In [ ]:
import pandas as pd
import numpy as np
import os
from statsmodels.stats.outliers_influence import variance_inflation_factor

eur_covar_path = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/real_data/UKB/pheno/covar/covars_white_british_qced.tsv"
eas_covar_path = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/real_data/UKB/pheno/covar/covars_chinese_qced.tsv"

eur_cov = pd.read_csv(eur_covar_path, sep="\t", header=0)
eas_cov = pd.read_csv(eas_covar_path, sep="\t", header=0)
eur_cov = eur_cov[["FID", "IID", "p31", "p21022", "p22009_a1", "p22009_a2", "p22009_a3", "p22009_a4", "p22009_a5", "p22009_a6", "p22009_a7", "p22009_a8", "p22009_a9", "p22009_a10", "p22009_a11", "p22009_a12", "p22009_a13", "p22009_a14", "p22009_a15", "p22009_a16", "p22009_a17", "p22009_a18", "p22009_a19", "p22009_a20"]]
eas_cov = eas_cov[["FID", "IID", "p31", "p21022", "p22009_a1", "p22009_a2", "p22009_a3", "p22009_a4", "p22009_a5", "p22009_a6", "p22009_a7", "p22009_a8", "p22009_a9", "p22009_a10"]]

# replace the column names
# p31 - sex; p22021 - age; p22009_a1 - PC1; p22009_a2 - PC2; p22009_a3 - PC3; p22009_a4 - PC4; p22009_a5 - PC5; p22009_a6 - PC6; p22009_a7 - PC7; p22009_a8 - PC8; p22009_a9 - PC9; p22009_a10 - PC10
eur_cov.columns = ["FID", "IID", "sex", "age", "PC1", "PC2", "PC3", "PC4", "PC5", "PC6", "PC7", "PC8", "PC9", "PC10", "PC11", "PC12", "PC13", "PC14", "PC15", "PC16", "PC17", "PC18", "PC19", "PC20"]
eas_cov.columns = ["FID", "IID", "sex", "age", "PC1", "PC2", "PC3", "PC4", "PC5", "PC6", "PC7", "PC8", "PC9", "PC10"]

# caculate other covariates
eur_cov["age2"] = eur_cov["age"] ** 2
eas_cov["age2"] = eas_cov["age"] ** 2
eur_cov["age_sex"] = eur_cov["age"] * eur_cov["sex"]
eas_cov["age_sex"] = eas_cov["age"] * eas_cov["sex"]
eur_cov["age2_sex"] = eur_cov["age2"] * eur_cov["sex"]
eas_cov["age2_sex"] = eas_cov["age2"] * eas_cov["sex"]

# save the covariates
eur_cols = ["FID", "IID", "age", "sex", "age2", "age_sex", "age2_sex"] + [f"PC{i}" for i in range(1, 21)]
eas_cols = ["FID", "IID", "age", "sex", "age2", "age_sex", "age2_sex"] + [f"PC{i}" for i in range(1, 11)]

# check if there are any missing values
if eur_cov.isnull().values.any():
    print("Warning: Missing values found in EUR covariates.")
if eas_cov.isnull().values.any():
    print("Warning: Missing values found in EAS covariates.")

def calculate_vif(df):
    """Calculate Variance Inflation Factor (VIF) for each feature in the DataFrame."""
    vif_data = pd.DataFrame()
    vif_data["feature"] = df.columns
    vif_data["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return vif_data

# Calculate VIF for EUR and EAS covariates
eur_vif = calculate_vif(eur_cov[eur_cols][2:])  # Exclude FID and IID
eas_vif = calculate_vif(eas_cov[eas_cols][2:])

# Print VIF results
print("EUR Covariates VIF:")
print(eur_vif)
print("\nEAS Covariates VIF:")
print(eas_vif)

eur_cov[eur_cols].to_csv("/data1/jiapl_group/lishuhua/project/PRS_benchmark/real_data/UKB/pheno/covar/covars_white_british_final.tsv", sep='\t', index=False)
eas_cov[eas_cols].to_csv("/data1/jiapl_group/lishuhua/project/PRS_benchmark/real_data/UKB/pheno/covar/covars_chinese_final.tsv", sep='\t', index=False)

In [ ]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor

eur_cov = pd.read_csv("/data1/jiapl_group/lishuhua/project/PRS_benchmark/real_data/UKB/pheno/covar/covars_white_british_final.tsv", sep="\t")
eas_cov = pd.read_csv("/data1/jiapl_group/lishuhua/project/PRS_benchmark/real_data/UKB/pheno/covar/covars_chinese_final.tsv", sep="\t")


def calculate_vif(df):
    """Calculate Variance Inflation Factor (VIF) for each feature in the DataFrame."""
    vif_data = pd.DataFrame()
    vif_data["feature"] = df.columns
    vif_data["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return vif_data

eur_cols_1 = ["age", "sex", "age2", "age_sex", "age2_sex"] +  [f'PC{i}' for i in range(1, 21)]
eur_cols_2 = ["age", "sex"] + [f'PC{i}' for i in range(1, 21)]
eas_cols_1 = ["age", "sex", "age2", "age_sex", "age2_sex"] +  [f'PC{i}' for i in range(1, 11)]
eas_cols_2 = ["age", "sex"] + [f'PC{i}' for i in range(1, 11)]


# Calculate VIF for EUR and EAS covariates
eur_vif_1 = calculate_vif(eur_cov[eur_cols_1])  # Exclude FID and IID
eas_vif_1 = calculate_vif(eas_cov[eas_cols_1])
eur_vif_2 = calculate_vif(eur_cov[eur_cols_2])  # Exclude FID and IID
eas_vif_2 = calculate_vif(eas_cov[eas_cols_2])

# Print VIF results
print("EUR Covariates VIF (type 1):")
print(eur_vif_1)
print("\nEAS Covariates VIF (type 1):")
print(eas_vif_1)

print("EUR Covariates VIF (type 2):")
print(eur_vif_2)
print("\nEAS Covariates VIF (type 2):")
print(eas_vif_2)